# *Pipeline de limpeza dos dados*

In [ ]:
#Resolvendo números em faixas --> acessar valor e substituir pela média da faixa
def calcular_media_faixa(valor):
    if isinstance(valor, str) and '-' in valor:
        try:
            # Divide a string em duas partes usando o '-' como separador
            parte1, parte2 = valor.split('-')
            # Converte as duas partes para float e calcula a média
            media = (float(parte1) + float(parte2)) / 2
            return media
        except ValueError:
            # Se algo der errado na conversão, retorna nulo
            return np.nan
    # Se não for uma faixa, retorna o valor como está
    return valor

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
df = pd.read_csv('imoveis_zap_fortaleza.csv',sep=";")
df.head()

In [ ]:
df_limpo = df.copy()

In [ ]:
df_limpo.info()

## *Coluna Preço*

In [ ]:
df_limpo['Preco'] = df_limpo['Preco'].str.replace(r'[^\d.]', '', regex=True) # r'[^\d.]'Remove tudo que NÃO é dígito (\d) ou ponto (.).
df_limpo['Preco'] = df_limpo['Preco'].str.replace('.','')
#Converte a coluna para um tipo numérico /  O 'errors='coerce'':se um valor não puder ser convertido como uma string vazia, ele será transformado em NaN (Not a Number)
df_limpo['Preco'] = pd.to_numeric(df_limpo['Preco'],errors='coerce')


## *Coluna Tamanho*

In [ ]:
df_limpo['Tamanho'] = df_limpo['Tamanho'].str.replace('Tamanho do imóvel','').str.replace('m²','')

In [ ]:
#Aplicação da função à coluna 'Tamanho' original
df_limpo['Tamanho_Limpo'] = df_limpo['Tamanho'].apply(calcular_media_faixa)
#Coluna seja numérica
df_limpo['Tamanho_Limpo'] = pd.to_numeric(df_limpo['Tamanho_Limpo'], errors='coerce')

## *Coluna Quartos*

In [ ]:
df_limpo['Quartos'] = df_limpo['Quartos'].str.replace('Quantidade de quartos','')

In [ ]:
#Aplicação da função à coluna 'Quartos' original --> porém será arrendado para o maior valor
df_limpo['Quartos_limpo'] = df_limpo['Quartos'].apply(calcular_media_faixa)
#Coluna seja numérica
df_limpo['Quartos_limpo'] = pd.to_numeric(df_limpo['Quartos_limpo'], errors='coerce')
df_limpo['Quartos_limpo'] = df_limpo['Quartos_limpo'].apply(lambda x: math.ceil(x) if pd.notna(x) else x) #Aplicando arredondamento

## *Coluna Banheiros*

In [ ]:
df_limpo['Banheiros'] = df_limpo['Banheiros'].str.replace('Quantidade de banheiros','')

In [ ]:
#Aplicação da função à coluna 'Quartos' original --> porém será arrendado para o maior valor
df_limpo['Banheiros_limpo'] = df_limpo['Banheiros'].apply(calcular_media_faixa)
#Coluna seja numérica
df_limpo['Banheiros_limpo'] = pd.to_numeric(df_limpo['Banheiros_limpo'], errors='coerce')
df_limpo['Banheiros_limpo'] = df_limpo['Banheiros_limpo'].apply(lambda x: math.ceil(x) if pd.notna(x) else x) #Aplicando arredondamento

## *Coluna Vagas*

In [ ]:
df_limpo['Vagas'] = df_limpo['Vagas'].str.replace('Quantidade de vagas de garagem','')

In [ ]:
#Aplicação da função à coluna 'Quartos' original --> porém será arrendado para o maior valor
df_limpo['Vagas_limpo'] = df_limpo['Vagas'].apply(calcular_media_faixa)
#Coluna seja numérica
df_limpo['Vagas_limpo'] = pd.to_numeric(df_limpo['Vagas_limpo'], errors='coerce')
df_limpo['Vagas_limpo'] = df_limpo['Vagas_limpo'].apply(lambda x: math.ceil(x) if pd.notna(x) else x) #Aplicando arredondamento

## *Coluna Bairro*

In [ ]:
df_limpo['Bairro'] = df_limpo['Bairro'].str.replace('Apartamento para comprar em','').str.replace(', Fortaleza','')

## *Dropando Colunas de apoio e renomeando*

In [ ]:
#Drop das colunas
df_limpo = df_limpo.drop(['Tamanho','Quartos','Banheiros','Vagas'],axis=1)

In [ ]:
#Renomeando colunas
df_limpo.rename(columns={'Tamanho_Limpo': 'tamanho (m²)',
                         'Quartos_limpo':'qtd_quartos',
                         'Banheiros_limpo':'qtd_banheiro',
                         'Vagas_limpo':'qtd_vagas'},
                           inplace=True)

# *Salvando base de dados final*

In [ ]:
df_limpo.info()

In [ ]:
#Salvando em arquivo CSV
nome_do_arquivo = 'imoveis_zap_fortaleza_ready.csv'
df_limpo.to_csv(nome_do_arquivo, index=False, sep=';', encoding='utf-8-sig')